# User-based Collaborative Filtering mit KNNWithMeans (Surprise)
Die bisher implementierten Collaborative Filtering Recommender Systeme (Aufgabe 5)basieren auf dem k-nearest neighbors (KNN)-Ansatz mit Cosine und Pearson Similarity sowie mittelwertzentrierten Nutzerbewertungen, wodurch bereits individuelle Bewertungstendenzen teilweise ausgeglichen werden.

Allerdings erfolgt die Umsetzung dort manuell, mit festem k-Wert und ohne systematische Optimierung. Um die Genauigkeit und Robustheit zu verbessern, verwende ich nun das Modell **KNNWithMeans** aus der surprise-Bibliothek.

Dieses Modell integriert die Mittelwertkorrektur direkt in den Algorithmus und ermöglicht zusätzlich eine automatische Optimierung des k-Werts per Cross-Validation. Dadurch wird nicht nur der Programmieraufwand reduziert, sondern auch die Vorhersagegenauigkeit gesteigert.

Ich gehe hier zur Demonstrierung lediglich auf user-based Collaborative Filtering mit Cosine Similarity ein und führe ein Hyperparametertuning durch, um das optimale k zu finden.

In [ ]:
from surprise import Dataset, Reader, KNNWithMeans
from surprise.model_selection import train_test_split, GridSearchCV
import numpy as np
from collections import defaultdict


# -----------------------------
# Daten vorbereiten
# -----------------------------
def prepare_data_knn(df):
    # Surprise benötigt ein spezielles Format. Der Reader definiert die Bewertungsskala
    reader = Reader(rating_scale=(0.5, 5))
    data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)
    trainset, testset = train_test_split(data, test_size=0.2, random_state=42)
    return data, trainset, testset

# -----------------------------
# Optimiertes KNN trainieren
# -----------------------------
def train_knn_optimized(data):
    # Suchraum für Hyperparameter: verschiedene Werte für k, cosine-Similarity, user-based
    param_grid = {
        'k': np.linspace(1, 50, 10, dtype=int), #10-Werte zwischen 1 und 50
        'sim_options': {
            'name': ['cosine'],
            'user_based': [True]
        }
    }
    # GridSearchCV zur automatischen Modell- und Parameteroptimierung anhand von RMSE über Cross-Validation
    gs = GridSearchCV(KNNWithMeans, param_grid, measures=['rmse'], cv=5, refit=True)
    gs.fit(data)

    # Ausgabe der besten Parameter und des besten RMSE-Werts
    best_k = gs.best_params['rmse']['k']
    best_rmse = np.round(gs.best_score['rmse'], 4)
    print(f"\nRMSE mit optimiertem K:")
    print(f"RMSE (k={best_k}): {best_rmse}")
    
    # Das beste Modell neu auf dem vollständigen Trainingsdatensatz trainieren um alle verfügbaren Trainingsdaten zu nutzen
    best_model = gs.best_estimator['rmse']
    best_model.fit(data.build_full_trainset())
    return best_model

# -----------------------------
# Evaluation: MAE / RMSE
# -----------------------------
def mean_absolute_error_knn(preds):
    true_ratings = [pred.r_ui for pred in preds]
    estimated_ratings = [pred.est for pred in preds]
    return np.mean(np.abs(np.array(true_ratings) - np.array(estimated_ratings)))

def root_mean_square_error_knn(preds):
    true_ratings = [pred.r_ui for pred in preds]
    estimated_ratings = [pred.est for pred in preds]
    return np.sqrt(np.mean((np.array(true_ratings) - np.array(estimated_ratings))**2))


def precision_recall_at_n_knn(predictions, threshold=4.0, n=15):
    top_n_pred = get_top_n(predictions, n=n)
    true_relevant = defaultdict(set)
    for pred in predictions:
        if pred.r_ui >= threshold:
            true_relevant[pred.uid].add(pred.iid)

    precisions, recalls = [], []
    for uid, user_preds in top_n_pred.items():
        recommended = set(iid for (iid, est) in user_preds if est >= threshold)
        relevant = true_relevant[uid]
        if not relevant:
            continue
        rel_and_rec = recommended & relevant
        precision = len(rel_and_rec) / len(recommended) if recommended else 0
        recall = len(rel_and_rec) / len(relevant)
        precisions.append(precision)
        recalls.append(recall)

    return np.mean(precisions), np.mean(recalls)

# -----------------------------
# Empfehlungen für Nutzer
# -----------------------------
def get_knn_recommendations(model, df, user_id, movies, n=20):
    rated_movies = df[df['userId'] == user_id]['movieId'].unique()
    all_movies = df['movieId'].unique()
    unrated_movies = [movie for movie in all_movies if movie not in rated_movies]

    predictions = []
    for movie_id in unrated_movies:
        pred = model.predict(user_id, movie_id)
        predictions.append((movie_id, pred.est))

    predictions.sort(key=lambda x: x[1], reverse=True)

    recommendations = f"\nTop {n} KNN-Empfehlungen für Benutzer {user_id}:\n"
    for i, (movie_id, rating) in enumerate(predictions[:n], 1):
        movie_title = movies.loc[movies['movieId'] == movie_id, 'title'].values[0]
        recommendations += f"{i}. {movie_title} - Vorhergesagte Bewertung: {rating}\n"

    return recommendations

In [ ]:
#Daten vorbereiten
data, trainset, testset = prepare_data_knn(ratings)

#Modell trainieren
knn_model = train_knn_optimized(data)

In [ ]:
#Vorhersagen
predictions = knn_model.test(testset)

#Metriken berechnen
mae = mean_absolute_error_knn(predictions)
rmse = root_mean_square_error_knn(predictions)
precision, recall = precision_recall_at_n_knn(predictions, n=15)

print("\nMetriken für optimiertes KNN-Modell:")
print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"Precision@15: {precision:.4f}")
print(f"Recall@15: {recall:.4f}")

In [ ]:
#Empfehlungen anzeigen
example_users = [3, 5, 7]  # ggf. anpassen
for user in example_users:
    print(get_knn_recommendations(knn_model, ratings, user, movies))